In [16]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st

In [17]:
filepath = "../main_resources/hc_workers/COVID19_ healthcare_workers.csv"
india_df = pd.read_csv(filepath, encoding='cp1252')

In [18]:
# Function to turn df.info into a dataframe for reference when cleaning data
def get_dataframe_info(df):
    """
    input
       df -> DataFrame
    output
       df_null_counts -> DataFrame Info (sorted)
    """

    df_types = pd.DataFrame(df.dtypes)
    df_nulls = df.count()
    
    df_null_count = pd.concat([df_types, df_nulls], axis=1)
    df_null_count = df_null_count.reset_index()
    
    # Reassign column names
    col_names = ["features", "types", "non_null_counts"]
    df_null_count.columns = col_names
    
    # Add this to sort
    #df_null_count = df_null_count.sort_values(by=["non_null_counts"], ascending=False)
    
    return df_null_count

In [19]:
# Write df.info to CSV
info_df = get_dataframe_info(india_df)
info_df.to_csv("og_info.csv", index=False, header=True)

In [20]:
# Drop last column since most people did not answer, and answers are custom inputs that are non-categorical
cleaned_df = india_df.drop("Is there anything else that you would like to share with regard to your mental health and well-being during the COVID-19 phase?",axis=1)

# Drop "What do you feel about your monetary situation during  the COVID 19 pandemic phase?". Not relevant to project
cleaned_df = cleaned_df.drop("What do you feel about your monetary situation during  the COVID 19 pandemic phase?", axis=1)

# Drop "With regard to travel to the hospital select that which suits you most" Not relevant to project
cleaned_df = cleaned_df.drop("With regard to travel to the hospital select that which suits you most", axis=1)

cleaned_df.shape

(240, 57)

In [21]:
# Slice out rows that have more than 30 NaN values (Did not answers more than half the questions)
cleaned_df = cleaned_df.loc[cleaned_df.isna().sum(axis=1)<=30,:]
cleaned_df.shape

(237, 57)

In [22]:
# Write cleaned dataframe.info to csv to check 
info2_df = get_dataframe_info(cleaned_df)
info2_df.to_csv("cleaned_info.csv", index=True, header=True)

# Write dataframe with any null values to CSV to view. Autofit columns Width and Go to special, highlight all blanks
cleaned_df.loc[cleaned_df.isna().sum(axis=1)>0,:].to_csv("df_null.csv", index=False, header=True)

In [23]:
# Get columns with any null values to discuss what to do (impute, drop, leave null)
test = cleaned_df.loc[cleaned_df.isna().sum(axis=1)>0,cleaned_df.isna().sum(axis=0)>0].columns.to_list()
x = pd.Series(test)
x.to_csv("columns_to_discuss.csv", index=False, header=False)

In [24]:
# # Combining similar values to clean up categories
# # Create a list of lists of each category per column

# answer_list = []
# for i in range(len(cleaned_df.columns)):
#     answer_list.append(cleaned_df.iloc[:,i].unique().tolist())

# # Check each column one by one, changing num index

# num = 17
# print(cleaned_df.columns[num])
# answer_list[num]

In [25]:
# Clean column 8 "During the lockdown period, indicate who you are living with:"
cleaned_df["During the lockdown period, indicate who you are living with:"] = cleaned_df["During the lockdown period, indicate who you are living with:"].replace({
    'Hostelers': 'Hospital Hostel',
    'Hostel': "Hospital Hostel",
    'With close family': 'Family',
    'With extended family': "Family",
    'TREATING COVID AND OTHER PATIENTS': "Hospital Hostel",
    'With hospital staff': "Hospital Hostel",
    'Co employers': "Hospital Hostel",
    'Stayed in a hospital,working there day and night for80days ': "Hospital Hostel"
})

cleaned_df.iloc[:,8].unique()

array(['Family', 'Alone', 'Friends', 'Hospital Hostel'], dtype=object)

In [26]:
# Clean column 17 "The hospital has provided me with adequate personal protective equipment"
cleaned_df["The hospital has provided me with adequate personal protective equipment"] = cleaned_df["The hospital has provided me with adequate personal protective equipment"].replace({
    'Hospital has tried its level best to provide but quite often things were not available in the market due to lockdown.': "No",
    'All except N 95 musk': "Yes",
    "My work as a teaching faculty member, I don't have to see patients.": "Not applicable",
    'Some': "Somewhat",
    'Not attached to any Hospital': "Not applicable",
    'I have arranged adequate  personal  protective  equipments. ': "Yes",
    'Retired. ': "Not applicable",
    "I don't fall into the category who needs to wear it necessarily. My hospital has provided PPE to all those who need it must.": "Yes",
    'I purchased  on my own. ': "No",
    'Not completely': "No",
    'N95 mask ,examination glove,head cap,handrub ,antiseptic solutions ': "Yes",
    'Mask and sanataoser': "Yes",
    'No need of PPE kit because i am in Audit dept': "Not applicable",
    'average protective ': "Somewhat",
    'Minimal assistance': "Somewhat",
    'Glowses': "Yes",
    'Not in contact with vivid+ patients': "Not applicable",
    'Not visiting hospital at present. ': "Not applicable",
    'Pps only': "Yes",
    'Not adequate initially, ok now': "Somewhat"
})

cleaned_df.iloc[:,17].unique()

array(['Yes', 'Somewhat', 'No', 'Not applicable', nan], dtype=object)

In [30]:
# Add ID columns so we can merge data on ID
cleaned_df = cleaned_df.reset_index()
cleaned_df["ID"] = cleaned_df.index

# Export cleaned dataframe to CSV
cleaned_df.to_csv("../main_resources/hc_workers/cleaned_hcw_df.csv", index=False, header=True)

In [28]:
# Break down col 17: "During the COVID-19 lockdown, I have received support from (you can select more than one answer)" into dataframe
social_support_series = cleaned_df["During the COVID-19 lockdown, I have received support from (you can select more than one answer)"]
social_support_df = social_support_series.str.get_dummies(sep=", ")

# Combine 
social_support_df["Government"] = social_support_df[['MD of my Hospital', 'My institute ICMR-RMRIMS ', 'police ,municipality ,medical college ']].max(axis=1)
# Drop 
social_support_df = social_support_df.drop(['MD of my Hospital', 'My institute ICMR-RMRIMS ', 'police ,municipality ,medical college '], axis=1)

# Combine 
social_support_df["Self"] = social_support_df[['Self', 'Not working as Healthcare ', 'Nil']].max(axis=1)
# Drop 
social_support_df = social_support_df.drop(['Not working as Healthcare ', 'Nil'], axis=1)

# Combine 
social_support_df['Wife/Husband'] = social_support_df[['Wife/Husband', 'Feoncy']].max(axis=1)
# Drop 
social_support_df = social_support_df.drop('Feoncy', axis=1)

# Combine 
social_support_df['Colleagues'] = social_support_df[['Colleagues', 'colleagues']].max(axis=1)
# Drop 
social_support_df = social_support_df.drop('colleagues', axis=1)


# Rename columns
social_support_df.columns = ['Sibling', 'Children', 'Colleagues',
                             'Extended family (relatives)', 'Parent', 
                             'Friends', 'Landlord', 'Self', 'Spouse', 'Government']


# Combine 
social_support_df['Family'] = social_support_df[['Sibling', 'Children', 'Extended family (relatives)', 'Parent']].max(axis=1)
# Drop 
social_support_df = social_support_df.drop(['Sibling', 'Children', 'Extended family (relatives)', 'Parent'], axis=1)

# Combine 
social_support_df['Friends'] = social_support_df[['Friends', 'Colleagues']].max(axis=1)
# Drop 
social_support_df = social_support_df.drop('Colleagues', axis=1)


social_support_df["ID"] = cleaned_df["ID"]
social_support_df.to_csv("../main_resources/hc_workers/received_support_from.csv", index=False, header=True)
social_support_df.shape

(237, 7)

In [29]:
# Break down col 18: "During the COVID-19 lockdown, I faced difficulties/problems with (you can select more than one answer)" into dataframe

problems_series = cleaned_df["During the COVID-19 lockdown, I faced difficulties/problems with (you can select more than one answer)"]

problems_df = problems_series.str.get_dummies(sep=", ")

# Combine 
problems_df["No problem"] = problems_df[["No problem", "Nil", "No", "No one", "No one ", "NIL", "No problem ", "No problems", "No problems ", "No problems faced", "No restraints faced", "Nobody", "None", "None ", "None. ", "Nothing ", "no one", "no problem", "none", "nothing.", "rather supportive environment.", "I did not face any issues", "I did not face significant problem ", "I have not faced any difficulty", "I stayed in company provided accommodation so no such problems", "Faced no problem with children."]].max(axis=1)
# Drop 
problems_df = problems_df.drop(["Nil", "No", "No one", "No one ", "NIL", "No problem ", "No problems", "No problems ", "No problems faced", "No restraints faced", "Nobody", "None", "None ", "None. ", "Nothing ", "no one", "no problem", "none", "nothing.", "rather supportive environment.", "I did not face any issues", "I did not face significant problem ", "I have not faced any difficulty", "I stayed in company provided accommodation so no such problems", "Faced no problem with children."], axis=1)

# Combine 
problems_df['Society / colony I live in'] = problems_df[["Society / colony I live in", "Faridabad and Government of Delhi", "Government", "Govt agencies due to conflicting orders of DM GB Nagar", "Faced problems getting 'out of order' things fixed and procure essential things. But not with people.", "My patients. ", "Naibers", "Other hospital staff", "Police", "Police because of pass", "Volunteers", 'Availability of trousers']].max(axis=1)
# Drop 
problems_df = problems_df.drop(["Faridabad and Government of Delhi", "Government", "Govt agencies due to conflicting orders of DM GB Nagar", "Faced problems getting 'out of order' things fixed and procure essential things. But not with people.", "My patients. ", "Naibers", "Other hospital staff", "Police", "Police because of pass", "Volunteers",'Availability of trousers'], axis=1)

problems_df.columns = ['Sibling', 'Children', 'Colleagues', 'Extended family', 'Parent', 'Friends', 'Landlord', 'No problem', 'Self', 'Society / colony I live in', 'Spouse']

# Combine 
problems_df['Family'] = problems_df[['Sibling', 'Children','Extended family', 'Parent']].max(axis=1)
# Drop 
problems_df = problems_df.drop(['Sibling', 'Children','Extended family', 'Parent'], axis=1)

# Combine 
problems_df['Friends'] = problems_df[['Friends', "Colleagues"]].max(axis=1)
# Drop 
problems_df = problems_df.drop("Colleagues", axis=1)


problems_df.columns = ['Friends', 'Landlord', 'None', 'Self', 'Government', 'Spouse', 'Family']


problems_df["ID"] = cleaned_df["ID"]
problems_df.to_csv("../main_resources/hc_workers/problems_with.csv", index=False, header=True)
problems_df.shape


(237, 8)